In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from sklearn.utils import shuffle

In [ ]:
from sklearn.model_selection import train_test_split
from keras import backend as K

In [ ]:
K.set_image_dim_ordering("tf")

In [ ]:
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense,Dropout,Activation,Flatten
from keras.layers.convolutional import Convolution2D,MaxPooling2D
from keras.optimizers import SGD,RMSprop,adam
from keras.callbacks import TensorBoard
from sklearn.metrics import classification_report,confusion_matrix
import itertools
import numpy as np

In [ ]:
PATH = os.getcwd()
data_path = 'input_1/data'
data_dir_list = os.listdir(data_path)


In [ ]:
data_dir_list

In [ ]:
img_rows=128

img_cols=128

num_channel=1

num_epoch=20

# Define the number of classes

num_classes = 7

img_data_list=[]

for dataset in data_dir_list:
    img_list=os.listdir(data_path+'/'+ dataset)
    print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
    for img in img_list:
        input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img )
        input_img=cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
        input_img_resize=cv2.resize(input_img,(128,128))
        img_data_list.append(input_img_resize)

img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
img_data /= 255
print (img_data.shape)


In [ ]:
if num_channel == 1:
    if K.image_dim_ordering() == "th":
        img_data = np.expand_dims(img_data,axis=1)
        print(img_data.shape)
    else:
        img_data = np.expand_dims(img_data,axis=4)
        print(img_data.shape)

else:
    if K.image_dim_ordering() == "th":
        img_data = np.rollaxis(img_data,3,1)
        print(img_data.shape)
    
    
    
    
    
    
    
    
    

In [ ]:
num_classes = 7
num_of_samples = img_data.shape[0]
labels = np.ones((num_of_samples,),dtype="int64")
labels[0:365]=0
labels[365:567]=1
labels[567:987]=2
labels[987:1189]=3
labels[1189:1399]=4
labels[1399:1601]=5
labels[1601:1803]=6

names = ['bike', 'cars', 'cats', 'dogs', 'flowers', 'horses', 'human']


In [ ]:
Y = np_utils.to_categorical(labels,num_classes)

In [ ]:
x,y = shuffle(img_data,Y,random_state=2)
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=2)

In [ ]:
print("X_train shape = {}".format(X_train.shape))
print("y_train shape = {}".format(y_train.shape))

In [ ]:
img = X_train[1201,:].reshape((128,128))
plt.imshow(img)
plt.show()

In [ ]:
input_shape=img_data[0].shape

In [ ]:
model = Sequential()

In [ ]:
model.add(Convolution2D(32,3,3,border_mode ='same',activation='relu',input_shape = input_shape))
model.add(Convolution2D(32,3,3,activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
model.add(Flatten())

model.add(Dense(128,activation="relu"))
model.add(Dropout(0.5))

model.add(Dense(num_classes,activation="softmax"))

In [ ]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
hist = model.fit(X_train,y_train,batch_size=16,nb_epoch=num_epoch,verbose=1,validation_data=(X_test,y_test))
train_loss = hist.history["loss"]
val_loss = hist.history["val_loss"]
train_acc = hist.history["acc"]
val_acc = hist.history["val_acc"]
xc = range(num_epoch)

In [ ]:
#Visualizing Training Loss & Validation Loss
plt.figure(1,figsize=(10,5))
plt.plot(xc,train_loss)
plt.plot(xc,val_loss)
plt.xlabel("Number of epochs")
plt.ylabel("Train Loass vs Validaion loss")
plt.grid(True)
plt.legend(["Train Loss","Validation Loss"])
plt.style.use(["classic"])

In [ ]:
#Visualizing Training Accuracy & Validation Accuracy
plt.figure(1,figsize=(10,5))
plt.plot(xc,train_acc)
plt.plot(xc,val_acc)
plt.xlabel("Number of epochs")
plt.ylabel("Accuracy")
plt.title("Train accuracy vs validation accuracy")
plt.grid(True)
plt.legend(["Train accuracy","Validation accuracy"],loc=4)
plt.style.use(["classic"])

In [ ]:
score=model.evaluate(X_test,y_test,verbose=8)
print("Test Loss:",score[0])
print("Test accuracy:",score[1])

In [ ]:
test_image = X_test[0:1]
print(test_image.shape)
print(model.predict(test_image))
print(model.predict_classes(test_image))
print(y_test[0:1])

In [ ]:
image = test_image.reshape((128,128))
plt.imshow(image)
plt.show()